## Backtesting smart money concept strategy

### Go to project root

In [1]:
import os
from pathlib import Path

#list the current work dir
cwd = os.getcwd()
current_path = Path(cwd)
project_root = current_path.parent.parent.parent

#change the current work dir
os.chdir(project_root)

In [2]:
%load_ext autoreload
%autoreload

In [ ]:
from trazy_analysis.common.backtest import Backtest, BacktestConfig
from trazy_analysis.broker.binance_fee_model import BinanceFeeModel
from trazy_analysis.models.asset import Asset
from trazy_analysis.strategy.strategies.smart_money_concept import (
    SmartMoneyConcept,
)
from trazy_analysis.models.enums import OrderType
from trazy_analysis.db_storage.influxdb_storage import InfluxDbStorage
from trazy_analysis.statistics.statistics import Statistics
from datetime import datetime, timedelta
import pytz

assets = [
    Asset(symbol="BTCUSDT", exchange="BINANCE", time_unit=timedelta(minutes=1)),
    #Asset(symbol="BTCUSDT", exchange="BINANCE", time_unit=timedelta(minutes=5)),
    #Asset(symbol="BTCUSDT", exchange="BINANCE", time_unit=timedelta(minutes=15)),
    #Asset(symbol="BTCUSDT", exchange="BINANCE", time_unit=timedelta(minutes=30)),
    #Asset(symbol="BTCUSDT", exchange="BINANCE", time_unit=timedelta(hours=1)),
    #Asset(symbol="BTCUSDT", exchange="BINANCE", time_unit=timedelta(hours=4))
]
start = datetime(2022, 6, 8, 0, 0, 0, 0, tzinfo=pytz.UTC)
end = datetime(2022, 6, 9, 0, 0, 0, 0, tzinfo=pytz.UTC)
db_storage = InfluxDbStorage()


backtest = Backtest(
    assets = assets,
    fee_model = BinanceFeeModel(),
    start=start,
    end=end,
    download=False,
    initial_funds = 10000.0,
    integer_size = False,
    fixed_order_type=OrderType.TARGET,
    target_order_pct=0.03,
    stop_order_pct=0.01,
    with_bracket=True,
    db_storage=db_storage,
    save_db_storage=None,
    preload = False,
    close_at_end_of_day=False,
    statistics_class = Statistics,
)

#results_df = backtest.run_strategy(SmartMoneyConcept, SmartMoneyConcept.DEFAULT_PARAMETERS)

In [ ]:
import numpy as np

results_df = backtest.run_strategy(
    SmartMoneyConcept,
    {
        "comparator": np.greater,
        "order": 3,
        "method": "local_extrema",
        "extrema_base": "body",
        "breakout_base": "body",
        "timeframe": timedelta(minutes=1),
    }
)

In [ ]:
import pandas
pandas.set_option('display.max_rows', None)

In [ ]:
backtest.backtest_config.feed.candle_dataframes

In [ ]:
backtest.event_loop.statistics_manager.transactions_dfs

In [ ]:
results_df.loc["Sortino Ratio"]["Backtest results"]

In [ ]:
results_df

In [ ]:
from trazy_analysis.optimization.optimization import Optimization
from trazy_analysis.optimization.optimizer import HyperOptimizer

optimizer = HyperOptimizer()
optimization = Optimization(
    backtest,
    optimizer,
    {SmartMoneyConcept: SmartMoneyConcept.DEFAULT_PARAMETERS_SPACE},
    nb_iter = 172
)

In [3]:
from trazy_analysis.common.live import Live
from trazy_analysis.models.asset import Asset
from trazy_analysis.strategy.strategies.smart_money_concept import (
    SmartMoneyConcept,
)
from trazy_analysis.models.enums import OrderType
from trazy_analysis.statistics.statistics import Statistics
from datetime import timedelta

assets = [
    Asset(symbol="BTCUSDT", exchange="binance", time_unit=timedelta(minutes=1)),
]

live = Live(
    assets = assets,
    integer_size = False,
    fixed_order_type=OrderType.TARGET,
    target_order_pct=0.03,
    stop_order_pct=0.01,
    with_bracket=True,
    preload = False,
    close_at_end_of_day=False,
    statistics_class = Statistics,
    simulate=True,
    simulation_initial_funds = 10000.0,
)

live.run_strategy(SmartMoneyConcept, SmartMoneyConcept.DEFAULT_PARAMETERS)

2022-06-10 15:32:43,199 - trazy_analysis.portfolio.portfolio - INFO - Portfolio "None" instance initialised
2022-06-10 15:32:43,201 - trazy_analysis.portfolio.portfolio - INFO - 2022-06-10 13:32:41.641345+00:00 - Funds subscribed to portfolio "None" - Credit: 0.0, Balance: 0.0
2022-06-10 15:32:43,202 - trazy_analysis.broker.broker - INFO - (2022-06-10 13:32:43.201975+00:00) - portfolio creation: Portfolio created
2022-06-10 15:32:43,202 - trazy_analysis.broker.simulated_broker - INFO - Initialising simulated broker...
2022-06-10 15:32:43,203 - trazy_analysis.portfolio.portfolio - INFO - 2022-06-10 13:32:41.641359+00:00 - Funds subscribed to portfolio "None" - Credit: 10000.0, Balance: 10000.0
2022-06-10 15:32:43,204 - trazy_analysis.broker.broker - INFO - (2022-06-10 13:32:43.204536+00:00) - subscription: 10000.0 subscribed to portfolio
2022-06-10 15:32:44,407 - trazy_analysis.bot.event_loop - INFO - Process new candle: {"asset": {"symbol": "BTCUSDT", "exchange": "binance", "time_unit"

2022-06-10 15:32:44,441 - trazy_analysis.bot.event_loop - INFO - binance: Strategy <trazy_analysis.strategy.strategies.smart_money_concept.SmartMoneyConcept object at 0x12d60f190> results: cash = 10000.0, portfolio = {}, total_equity = 10000.0
2022-06-10 15:32:44,443 - trazy_analysis.bot.event_loop - INFO - Process new candle: {"asset": {"symbol": "BTCUSDT", "exchange": "binance", "time_unit": "0:01:00"}, "open": "30139.8", "high": "30159.98", "low": "30139.79", "close": "30159.97", "volume": 8.62362, "timestamp": "2022-06-10 05:27:00+0000"}
2022-06-10 15:32:44,445 - trazy_analysis.indicators.bos - INFO - Waiting for BOS 0:01:00
2022-06-10 15:32:44,446 - trazy_analysis.bot.event_loop - INFO - binance: Strategy <trazy_analysis.strategy.strategies.smart_money_concept.SmartMoneyConcept object at 0x12d60f190> results: cash = 10000.0, portfolio = {}, total_equity = 10000.0
2022-06-10 15:32:44,449 - trazy_analysis.bot.event_loop - INFO - Process new candle: {"asset": {"symbol": "BTCUSDT", "e

2022-06-10 15:32:44,495 - trazy_analysis.indicators.bos - INFO - Waiting for BOS 0:01:00
2022-06-10 15:32:44,496 - trazy_analysis.bot.event_loop - INFO - binance: Strategy <trazy_analysis.strategy.strategies.smart_money_concept.SmartMoneyConcept object at 0x12d60f190> results: cash = 10000.0, portfolio = {}, total_equity = 10000.0
2022-06-10 15:32:44,497 - trazy_analysis.bot.event_loop - INFO - Process new candle: {"asset": {"symbol": "BTCUSDT", "exchange": "binance", "time_unit": "0:01:00"}, "open": "30171.45", "high": "30171.45", "low": "30167.83", "close": "30167.83", "volume": 3.91861, "timestamp": "2022-06-10 05:41:00+0000"}
2022-06-10 15:32:44,499 - trazy_analysis.bot.event_loop - INFO - binance: Strategy <trazy_analysis.strategy.strategies.smart_money_concept.SmartMoneyConcept object at 0x12d60f190> results: cash = 10000.0, portfolio = {}, total_equity = 10000.0
2022-06-10 15:32:44,500 - trazy_analysis.bot.event_loop - INFO - Process new candle: {"asset": {"symbol": "BTCUSDT", "

2022-06-10 15:32:44,557 - trazy_analysis.bot.event_loop - INFO - Process new candle: {"asset": {"symbol": "BTCUSDT", "exchange": "binance", "time_unit": "0:01:00"}, "open": "30110.45", "high": "30125.4", "low": "30110.04", "close": "30125.39", "volume": 19.05409, "timestamp": "2022-06-10 05:55:00+0000"}
2022-06-10 15:32:44,560 - trazy_analysis.indicators.bos - INFO - Waiting for BOS 0:01:00
2022-06-10 15:32:44,565 - trazy_analysis.bot.event_loop - INFO - binance: Strategy <trazy_analysis.strategy.strategies.smart_money_concept.SmartMoneyConcept object at 0x12d60f190> results: cash = 10000.0, portfolio = {}, total_equity = 10000.0
2022-06-10 15:32:44,570 - trazy_analysis.bot.event_loop - INFO - Process new candle: {"asset": {"symbol": "BTCUSDT", "exchange": "binance", "time_unit": "0:01:00"}, "open": "30125.4", "high": "30125.4", "low": "30113.79", "close": "30125.14", "volume": 9.66221, "timestamp": "2022-06-10 05:56:00+0000"}
2022-06-10 15:32:44,573 - trazy_analysis.bot.event_loop - I

2022-06-10 15:32:44,624 - trazy_analysis.bot.event_loop - INFO - binance: Strategy <trazy_analysis.strategy.strategies.smart_money_concept.SmartMoneyConcept object at 0x12d60f190> results: cash = 10000.0, portfolio = {}, total_equity = 10000.0
2022-06-10 15:32:44,626 - trazy_analysis.bot.event_loop - INFO - Process new candle: {"asset": {"symbol": "BTCUSDT", "exchange": "binance", "time_unit": "0:01:00"}, "open": "30106.99", "high": "30108.48", "low": "30106.98", "close": "30108.48", "volume": 6.08338, "timestamp": "2022-06-10 06:10:00+0000"}
2022-06-10 15:32:44,628 - trazy_analysis.bot.event_loop - INFO - binance: Strategy <trazy_analysis.strategy.strategies.smart_money_concept.SmartMoneyConcept object at 0x12d60f190> results: cash = 10000.0, portfolio = {}, total_equity = 10000.0
2022-06-10 15:32:44,629 - trazy_analysis.bot.event_loop - INFO - Process new candle: {"asset": {"symbol": "BTCUSDT", "exchange": "binance", "time_unit": "0:01:00"}, "open": "30108.47", "high": "30174.85", "l

...

2022-06-10 15:32:45,642 - trazy_analysis.bot.event_loop - INFO - Process new candle: {"asset": {"symbol": "BTCUSDT", "exchange": "binance", "time_unit": "0:01:00"}, "open": "29556.99", "high": "29619.05", "low": "29511.4", "close": "29553.89", "volume": 462.18946, "timestamp": "2022-06-10 12:34:00+0000"}
2022-06-10 15:32:45,643 - trazy_analysis.bot.event_loop - INFO - binance: Strategy <trazy_analysis.strategy.strategies.smart_money_concept.SmartMoneyConcept object at 0x12d60f190> results: cash = 10000.0, portfolio = {}, total_equity = 10000.0
2022-06-10 15:32:45,644 - trazy_analysis.bot.event_loop - INFO - Process new candle: {"asset": {"symbol": "BTCUSDT", "exchange": "binance", "time_unit": "0:01:00"}, "open": "29553.89", "high": "29699.24", "low": "29505.53", "close": "29671.95", "volume": 347.07063, "timestamp": "2022-06-10 12:35:00+0000"}
2022-06-10 15:32:45,644 - trazy_analysis.bot.event_loop - INFO - binance: Strategy <trazy_analysis.strategy.strategies.smart_money_concept.Smar

2022-06-10 15:32:45,671 - trazy_analysis.bot.event_loop - INFO - binance: Strategy <trazy_analysis.strategy.strategies.smart_money_concept.SmartMoneyConcept object at 0x12d60f190> results: cash = 10000.0, portfolio = {}, total_equity = 10000.0
2022-06-10 15:32:45,671 - trazy_analysis.bot.event_loop - INFO - Process new candle: {"asset": {"symbol": "BTCUSDT", "exchange": "binance", "time_unit": "0:01:00"}, "open": "29756.41", "high": "29798.86", "low": "29751.53", "close": "29751.53", "volume": 85.02691, "timestamp": "2022-06-10 12:49:00+0000"}
2022-06-10 15:32:45,672 - trazy_analysis.bot.event_loop - INFO - binance: Strategy <trazy_analysis.strategy.strategies.smart_money_concept.SmartMoneyConcept object at 0x12d60f190> results: cash = 10000.0, portfolio = {}, total_equity = 10000.0
2022-06-10 15:32:45,673 - trazy_analysis.bot.event_loop - INFO - Process new candle: {"asset": {"symbol": "BTCUSDT", "exchange": "binance", "time_unit": "0:01:00"}, "open": "29751.53", "high": "29767.41", "

2022-06-10 15:32:45,702 - trazy_analysis.bot.event_loop - INFO - Process new candle: {"asset": {"symbol": "BTCUSDT", "exchange": "binance", "time_unit": "0:01:00"}, "open": "29660.48", "high": "29691.91", "low": "29651.1", "close": "29690.31", "volume": 57.57256, "timestamp": "2022-06-10 13:03:00+0000"}
2022-06-10 15:32:45,703 - trazy_analysis.bot.event_loop - INFO - binance: Strategy <trazy_analysis.strategy.strategies.smart_money_concept.SmartMoneyConcept object at 0x12d60f190> results: cash = 10000.0, portfolio = {}, total_equity = 10000.0
2022-06-10 15:32:45,705 - trazy_analysis.bot.event_loop - INFO - Process new candle: {"asset": {"symbol": "BTCUSDT", "exchange": "binance", "time_unit": "0:01:00"}, "open": "29690.3", "high": "29711.69", "low": "29653.57", "close": "29711.68", "volume": 96.48201, "timestamp": "2022-06-10 13:04:00+0000"}
2022-06-10 15:32:45,706 - trazy_analysis.bot.event_loop - INFO - binance: Strategy <trazy_analysis.strategy.strategies.smart_money_concept.SmartMo

2022-06-10 15:32:45,731 - trazy_analysis.bot.event_loop - INFO - binance: Strategy <trazy_analysis.strategy.strategies.smart_money_concept.SmartMoneyConcept object at 0x12d60f190> results: cash = 10000.0, portfolio = {}, total_equity = 10000.0
2022-06-10 15:32:45,732 - trazy_analysis.bot.event_loop - INFO - Process new candle: {"asset": {"symbol": "BTCUSDT", "exchange": "binance", "time_unit": "0:01:00"}, "open": "29601.46", "high": "29631.28", "low": "29595.43", "close": "29625.7", "volume": 53.95699, "timestamp": "2022-06-10 13:18:00+0000"}
2022-06-10 15:32:45,733 - trazy_analysis.bot.event_loop - INFO - binance: Strategy <trazy_analysis.strategy.strategies.smart_money_concept.SmartMoneyConcept object at 0x12d60f190> results: cash = 10000.0, portfolio = {}, total_equity = 10000.0
2022-06-10 15:32:45,734 - trazy_analysis.bot.event_loop - INFO - Process new candle: {"asset": {"symbol": "BTCUSDT", "exchange": "binance", "time_unit": "0:01:00"}, "open": "29625.7", "high": "29625.7", "low

KeyboardInterrupt: 